In [1]:
import VGG as vgg
import torch
from HSI_class import HSI
import numpy as np

import os

directory = r"C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune"
files = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# directory2 = r"C:\Users\Asus TUF\Documents\code\TA\models\model_800train"
# files2 = [os.path.join(directory2, f) for f in os.listdir(directory2) if os.path.isfile(os.path.join(directory2, f))]


models = []

for file in files:
    models.append(file)

# for file in files2:
#     models.append(file)

for model in models:
    print(model)


C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\checkpoint_0001.pth.tar
C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\checkpoint_0002.pth.tar
C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\checkpoint_0003.pth.tar
C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\checkpoint_0004.pth.tar
C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\checkpoint_0005.pth.tar
C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\checkpoint_0006.pth.tar
C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\checkpoint_0007.pth.tar
C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\checkpoint_0008.pth.tar
C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\checkpoint_0009.pth.tar
C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\checkpoint_0010.pth.tar


In [2]:
dataset_path = "Hyperspectral oil spill detection datasets"

dataset = []

i = 0
for filename in os.listdir(dataset_path):
    if i > 3:
        break
    file_path = os.path.join(dataset_path, filename)
    if os.path.isfile(file_path):  # Check if it's a file
        print(f"Processing file: {file_path}")
        hsi = HSI(file_path)
        dataset.append(hsi)
    i += 1

Processing file: Hyperspectral oil spill detection datasets\GM01.mat
Processing file: Hyperspectral oil spill detection datasets\GM02.mat
Processing file: Hyperspectral oil spill detection datasets\GM03.mat
Processing file: Hyperspectral oil spill detection datasets\GM04.mat


In [3]:
import zeroPadding
hsi_test = dataset[2]

test_img = hsi_test.img
test_gt = hsi_test.gt

patch_size = 9
half_patch = patch_size // 2

height = test_img.shape[0]
width = test_img.shape[1]

matrix=zeroPadding.zeroPadding_3D(test_img,half_patch) #add 0 in every side of the data
print(f"img shape: {test_img.shape}")
print(f"img shape after padding {matrix.shape}")
print(f"number of pixel {width * height}")

img shape: (1386, 690, 224)
img shape after padding (1394, 698, 224)
number of pixel 956340


In [4]:
print(test_gt.shape)

indices0 = np.argwhere(test_gt == 0)
indices1 = np.argwhere(test_gt == 1)

print(indices0.shape)
print(indices1.shape)

num_samples = 100

random_indices0 = indices0[np.random.choice(len(indices0), num_samples, replace=False)]
random_indices1 = indices1[np.random.choice(len(indices1), num_samples, replace=False)]

test_indices = np.vstack((random_indices0, random_indices1))

print(test_indices.shape)

(1386, 690)
(916980, 2)
(39360, 2)
(200, 2)


In [5]:
def predict(saved_model, input):

    with torch.no_grad():  # Disable gradients for inference
        output = saved_model(input)

    # Convert logits to class label
    predicted_class = torch.argmax(output, dim=1).item()
    confidence = torch.nn.functional.softmax(output, dim=1)[0, predicted_class].item()

    return predicted_class, confidence

In [6]:
from torchvision.models import vgg16
import torch.nn as nn

class VGG16_HSI(nn.Module):
    def __init__(self, num_classes=2):
        super(VGG16_HSI, self).__init__()

         # Custom Convolutional Layer: Process 9x9x224 input
        self.pre_conv = nn.Sequential(
            nn.Conv2d(in_channels=224, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.AdaptiveAvgPool2d((1, 1))  # Reduce to (256, 1, 1)
        )

        # Fully Connected Layer to reshape to (64, 56, 56)
        self.fc = nn.Linear(256 * 1 * 1, 64 * 56 * 56)

        # Load VGG-16 Model
        self.encoder = vgg16(pretrained=False)

        # Remove first VGG-16 conv layer
        self.encoder.features = nn.Sequential(*list(self.encoder.features.children())[1:])

        # Modify classifier to output 2 classes
        self.encoder.classifier[6] = nn.Linear(4096, 2)

    def forward(self, x):
        # print(f'before {x.shape}')
        x = self.pre_conv(x)  # Process hyperspectral input
        x = x.view(x.size(0), -1)  # Flatten

        # print(f'after preconv {x.shape}')
        x = self.fc(x)  # Fully connected layer
        # print(f'after fc {x.shape}')
        # Reshape to (batch_size, 64, 56, 56) before passing to VGG
        x = x.view(x.size(0), 64, 56, 56)
        # print(f'after reshape, before vgg second layer {x.shape}')

        x = self.encoder.features(x)  # Pass to VGG-16
        x = self.encoder.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.encoder.classifier(x)  # Final classification layer

        return x

In [16]:
scores = []

for model in models:
    print("creating model...")
    saved_model = VGG16_HSI()
    checkpoint = torch.load(model)
    # print(checkpoint)
    saved_model.load_state_dict(checkpoint['state_dict'])
    print("model created")

    total = 0
    correct = 0

    i = 0
    for indice in test_indices:
        x_pos = indice[0]
        y_pos = indice[1]

        true_label = test_gt[x_pos][y_pos]

        selected_rows = matrix[range(x_pos,x_pos+2*half_patch+1), :]
        testing_patch = selected_rows[:, range(y_pos, y_pos+2*half_patch+1)]
        
        # print(i)
        # print(testing_patch[half_patch][half_patch])
        # print(test_img[x_pos][y_pos])

        testing_patch = torch.tensor(testing_patch)
        testing_patch = testing_patch.to(torch.float32)
        testing_patch = testing_patch.unsqueeze(0)
        testing_patch = testing_patch.permute(0, 3, 1, 2)

        prediction, confidence = predict(saved_model, testing_patch)

        print(f"{i+1}: prediction = {prediction}, confidence: {confidence}, expected: {true_label}")
        
        if(prediction == true_label):
            correct += 1

        total += 1
        i += 1

    print(f"skor: {correct}/{total}")
    model_name = model.split('\\')
    scores.append((model_name[10], f'{correct}/{total}'))
 


creating model...


C:\Users\Asus TUF\AppData\Local\Temp\ipykernel_6636\684308176.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model)


model created
1: prediction = 0, confidence: 0.5697548389434814, expected: 0
2: prediction = 1, confidence: 0.691260576248169, expected: 0
3: prediction = 1, confidence: 0.7539708018302917, expected: 0
4: prediction = 1, confidence: 0.599081814289093, expected: 0
5: prediction = 0, confidence: 0.6671732664108276, expected: 0
6: prediction = 1, confidence: 0.7637499570846558, expected: 0
7: prediction = 1, confidence: 0.6824236512184143, expected: 0
8: prediction = 0, confidence: 0.6046593189239502, expected: 0
9: prediction = 1, confidence: 0.5638601779937744, expected: 0
10: prediction = 0, confidence: 0.6246122717857361, expected: 0
11: prediction = 0, confidence: 0.5144935250282288, expected: 0
12: prediction = 0, confidence: 0.6040345430374146, expected: 0
13: prediction = 1, confidence: 0.5238881707191467, expected: 0
14: prediction = 1, confidence: 0.7492715120315552, expected: 0
15: prediction = 1, confidence: 0.5202909708023071, expected: 0
16: prediction = 0, confidence: 0.625

In [17]:
for score in scores:
    print(score)

('checkpoint_0001.pth.tar', '103/200')
('checkpoint_0002.pth.tar', '89/200')
('checkpoint_0003.pth.tar', '98/200')
('checkpoint_0004.pth.tar', '101/200')
('checkpoint_0005.pth.tar', '98/200')
('checkpoint_0006.pth.tar', '100/200')
('checkpoint_0007.pth.tar', '100/200')
('checkpoint_0008.pth.tar', '100/200')
('checkpoint_0009.pth.tar', '100/200')
('checkpoint_0010.pth.tar', '100/200')


In [ ]:
('checkpoint_0001.pth.tar', '103/200')
('checkpoint_0002.pth.tar', '89/200')
('checkpoint_0003.pth.tar', '98/200')
('checkpoint_0004.pth.tar', '101/200')
('checkpoint_0005.pth.tar', '98/200')
('checkpoint_0006.pth.tar', '100/200')
('checkpoint_0007.pth.tar', '100/200')
('checkpoint_0008.pth.tar', '100/200')
('checkpoint_0009.pth.tar', '100/200')
('checkpoint_0010.pth.tar', '100/200')